In [ ]:
import pandas as pd
import numpy as np

In [ ]:
file_path = '../Jeux de données-20260109/Flights_20191201_20191231.csv/Flights_20191201_20191231.csv'

# Chargement avec paramètres spécifiques
df_Flights_20191201_20191231 = pd.read_csv(
                                file_path,
                                sep=',',              # Définit le séparateur (souvent ; en France)
                                encoding='utf-8',     # Encodage (essayer 'latin1' ou 'cp1252' si erreur)
                                index_col=0,          # Utilise la 1ère colonne comme index (optionnel)
                                na_values=['n/a', '?'] # Convertit ces valeurs spécifiques en NaN (vide)
                            )


In [ ]:
def select_air_france(df): 
    df_select = df[df['AC Operator'] == 'AFR']

    print("Nombre de vols air france: ", len(df_select))
    return df_select

def select_europe(df): 
    codes_europe = ['L', 'E', 'B']

    # 3. On crée un filtre : Le 1er caractère de ADEP *ET* de ADES doit être dans la liste
    # .str[0] permet de prendre juste la première lettre de la colonne
    filtre_europe = (
        df['ADEP'].str[0].isin(codes_europe) & 
        df['ADES'].str[0].isin(codes_europe)
    )

    # 4. On applique le filtre
    vols_europe = df[filtre_europe]

    print(f"Nombre de vols trouvés : {len(vols_europe)}")
    print(vols_europe[['ADEP', 'ADES']].head())

    return vols_europe

# def top_flights(df, n_top = 100): 
#     classement_liaisons = df.groupby(['ADEP', 'ADES']).size()
#     classement_liaisons = classement_liaisons.sort_values(ascending=False)
#     classement_liaisons = classement_liaisons.reset_index(name='Nombre_de_vols')
#     # Affichage du TOP 10
#     print("Top 10 des liaisons Air France en Europe :")
#     print(classement_liaisons.head(n_top))

#     return classement_liaisons.head(n_top)

def top_flights(df, n_top=100):
    df_temp = df.copy()

    routes_triees = np.sort(df_temp[['ADEP', 'ADES']].values, axis=1)

    df_temp['Aeroport_1'] = routes_triees[:, 0]
    df_temp['Aeroport_2'] = routes_triees[:, 1]

    classement_liaisons = df_temp.groupby(['Aeroport_1', 'Aeroport_2']).size()

    classement_liaisons = classement_liaisons.sort_values(ascending=False)
    classement_liaisons = classement_liaisons.reset_index(name='Nombre_de_vols')


    print(f"Top {n_top} des liaisons (Sens confondus) :")
    print(classement_liaisons.head(n_top))

    return classement_liaisons.head(n_top)

# def selec_top_flights(df, top_flight): 
#     clés_top_10 = top_flight[['Aeroport_1', 'Aeroport_2']] + top_flight[['Aeroport_2', 'Aeroport_1']]

#     masque_filtre = (
#         df.set_index(['ADEP', 'ADES']).index.isin(clés_top_10.set_index(['ADEP', 'ADES']).index)
#     )

#     # 3. On applique ce masque au DataFrame original
#     vols_du_top = df[masque_filtre]

#     # Vérification
#     print("Nombre de vols retenus : ", len(vols_du_top))

#     return vols_du_top

def selec_top_flights(df, top_flight):
    # 1. Préparation de la liste complète des couples autorisés (A->B ET B->A)
    
    # On prend le sens "Aller" (tel quel dans le top) et on renomme pour matcher df
    sens_aller = top_flight[['Aeroport_1', 'Aeroport_2']].rename(
        columns={'Aeroport_1': 'ADEP', 'Aeroport_2': 'ADES'}
    )
    
    # On prend le sens "Retour" (on inverse les colonnes) et on renomme pour matcher df
    sens_retour = top_flight[['Aeroport_2', 'Aeroport_1']].rename(
        columns={'Aeroport_2': 'ADEP', 'Aeroport_1': 'ADES'}
    )
    
    # On concatène les deux pour avoir une liste exhaustive de toutes les paires valides
    toutes_les_cles = pd.concat([sens_aller, sens_retour])

    # 2. Création du masque
    # On vérifie si le couple (ADEP, ADES) du vol existe dans notre liste exhaustive
    masque_filtre = (
        df.set_index(['ADEP', 'ADES']).index.isin(toutes_les_cles.set_index(['ADEP', 'ADES']).index)
    )

    # 3. On applique ce masque au DataFrame original
    vols_du_top = df[masque_filtre]

    # Vérification
    print(f"Nombre de vols retenus : {len(vols_du_top)} (sur {len(df)})")

    return vols_du_top

In [ ]:
print("Nombre de vols décembre en Europe :", len(df_Flights_20191201_20191231))
print("\n")
vols_af = select_air_france(df_Flights_20191201_20191231)
print("\n")
vols_af_europe = select_europe(vols_af)
print("\n")
top_lines = top_flights(vols_af_europe, 20)
print("\n")
top_vols_af_europe = selec_top_flights(vols_af_europe, top_lines)

Nombre de vols décembre en Europe : 747570


Nombre de vols air france:  30421


Nombre de vols trouvés : 24244
           ADEP  ADES
ECTRL ID             
236565987  LTFM  LFPG
236566467  LROP  LFPG
236566523  LGAV  LFPG
236566830  LFML  LFPG
236566832  LHBP  LFPG


Top 20 des liaisons (Sens confondus) :
   Aeroport_1 Aeroport_2  Nombre_de_vols
0        LFBO       LFPO             940
1        LFMN       LFPO             750
2        LFML       LFPO             603
3        LFPG       LSGG             522
4        LFMT       LFPO             490
5        LFBD       LFPO             475
6        LFMN       LFPG             434
7        LFBO       LFPG             419
8        LEBL       LFPG             414
9        LFML       LFPG             381
10       EGLL       LFPG             366
11       LEMD       LFPG             364
12       EHAM       LFPG             360
13       LFBD       LFPG             356
14       LFPG       LIRF             353
15       EDDT       LFPG             

In [ ]:
top_vols_af_europe['AC Type'].unique()

array(['A319', 'A318', 'A320', 'A321', 'E190', 'CRJ7', 'CRJX', 'E170',
       'E145', 'B789'], dtype=object)

In [ ]:
# Dictionnaire de correspondance (OACI -> Ville)
dico_villes = {
    'LFBO': 'Toulouse', 'LFPO': 'Paris-Orly', 'LFMN': 'Nice', 
    'LFML': 'Marseille', 'LFPG': 'Paris-CDG', 'LSGG': 'Genève', 
    'LFMT': 'Montpellier', 'LFBD': 'Bordeaux', 'LEBL': 'Barcelone', 
    'EGLL': 'Londres-Heathrow', 'LEMD': 'Madrid', 'EHAM': 'Amsterdam', 
    'LIRF': 'Rome', 'EDDT': 'Berlin', 'EDDM': 'Munich', 
    'LFLL': 'Lyon', 'EDDF': 'Francfort', 'LFTH': 'Toulon'
}

# Supposons que votre dataframe de résultat s'appelle 'top_20'
# On ajoute les colonnes Ville
top_lines['Ville_1'] = top_lines['Aeroport_1'].map(dico_villes)
top_lines['Ville_2'] = top_lines['Aeroport_2'].map(dico_villes)

# On affiche pour vérifier
print(top_lines[['Ville_1', 'Ville_2', 'Nombre_de_vols']])

             Ville_1     Ville_2  Nombre_de_vols
0           Toulouse  Paris-Orly             940
1               Nice  Paris-Orly             750
2          Marseille  Paris-Orly             603
3          Paris-CDG      Genève             522
4        Montpellier  Paris-Orly             490
5           Bordeaux  Paris-Orly             475
6               Nice   Paris-CDG             434
7           Toulouse   Paris-CDG             419
8          Barcelone   Paris-CDG             414
9          Marseille   Paris-CDG             381
10  Londres-Heathrow   Paris-CDG             366
11            Madrid   Paris-CDG             364
12         Amsterdam   Paris-CDG             360
13          Bordeaux   Paris-CDG             356
14         Paris-CDG        Rome             353
15            Berlin   Paris-CDG             346
16            Munich   Paris-CDG             324
17              Lyon   Paris-CDG             309
18         Francfort   Paris-CDG             297
19        Paris-Orly